In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
base_model = "jgchaparro/llama-tsakonian-test"
tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Run text generation pipeline with our next model
prompt = "Τσ̇ι έν̇ι τα Τσακώνικα;"
prompt = "Τσ̇'εν̇ι μάθου;"
pipe = pipeline(task="text-generation", model=model, tokenizer = tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} Ένι μάθου [/INST]")
print(result[0]['generated_text'])

<s>[INST] Τσ̇'εν̇ι μάθου; Ένι μάθου [/INST]  The Greek language has a complex system of noun classification, with nouns being grouped into several categories based on their grammatical properties. The most common classification of nouns in Greek is the following:

1. Ένι (Énsi) - This category includes nouns that refer to living things, such as animals, plants, and people. Examples: ενόπlo, ένι, ένια.
2. Τσαγλασ [θ] (Tsaglās) - This category includes nouns that refer to non-living things, such as objects, places, and concepts. Examples: τσαγλασ, τσα


In [ ]:
prompt = "Τσ̇ι έν̇ι τα Τσακώνικα;"
refined_prompt = f"<s>[INST] {prompt} [/INST]"

In [ ]:
model(refined_prompt)

AttributeError: ignored